# Quantum Teleportation 

Here we will try to teleport $$\sqrt{0.70}\vert0\rangle + \sqrt{0.30}\vert1\rangle$$ from Alice's qubit to Bob's qubit. 
 
The teleportation algorithm is as follows: 
1. Initializing the state to be teleported. We will do this on Alice's qubit `q0`.
2. Creating entanglement between two qubits. We will use qubits `q1` and `q2` for this.
3. Applying a Bell measurement on Alice's qubits `q0` and `q1`.
4. Applying classically controlled operations on Bob's qubit `q2` depending on the outcomes of the Bell measurement on Alice's qubits.

In [13]:
#imports 
!pip install -U -r resources/requirements.txt

from IPython.display import clear_output
clear_output()

## Initializing the state to be teleported 

In [20]:
def initialize_qubit(given_circuit, qubit_index):
    import numpy as np
    given_circuit.initialize([np.sqrt(0.7),np.sqrt(0.3)],qubit_index)
    return given_circuit

## Creating entaglement between Alice and Bob's qubits

In [21]:
def entangle_qubits(given_circuit, qubit_Alice, qubit_Bob):
    given_circuit.h(qubit_Alice)
    given_circuit.cx(qubit_Alice, qubit_Bob)
    return given_circuit

## Bell Measurement of Alice's qubits

In [22]:
def bell_meas_Alice_qubits(given_circuit, qubit1_Alice, qubit2_Alice, clbit1_Alice, clbit2_Alice):
    given_circuit.cx(qubit1_Alice, qubit2_Alice)
    given_circuit.h(qubit1_Alice)
    given_circuit.barrier()
    given_circuit.measure(qubit1_Alice, clbit1_Alice)
    given_circuit.measure(qubit2_Alice, clbit2_Alice)
    return given_circuit

## Applying controlled operations 
We can now apply controlled operations on Bob's qubit. The operations are: 

- an $X$ gate is applied on Bob's qubit if the measurement coutcome of Alice's second qubit, `clbit2_Alice`, is `1`.
- a $Z$ gate is applied on Bob's qubit if the measurement coutcome of Alice's first qubit, `clbit1_Alice`, is `1`.

In [23]:
def controlled_ops_Bob_qubit(given_circuit, qubit_Bob, clbit1_Alice, clbit2_Alice):
    given_circuit.x(qubit_Bob).c_if(clbit2_Alice, 1)
    given_circuit.z(qubit_Bob).c_if(clbit1_Alice, 1)    
    return given_circuit

In [24]:
from qiskit import QuantumCircuit


In [25]:
### imports
from qiskit import QuantumRegister, ClassicalRegister

### set up the qubits and classical bits
all_qubits_Alice = QuantumRegister(2)
all_qubits_Bob = QuantumRegister(1)
creg1_Alice = ClassicalRegister(1)
creg2_Alice = ClassicalRegister(1)

### quantum teleportation circuit here
# Initialize
mycircuit = QuantumCircuit(all_qubits_Alice, all_qubits_Bob, creg1_Alice, creg2_Alice)
initialize_qubit(mycircuit, 0)
mycircuit.barrier()
# Entangle
entangle_qubits(mycircuit, 1, 2)
mycircuit.barrier()
# Do a Bell measurement
bell_meas_Alice_qubits(mycircuit, all_qubits_Alice[0], all_qubits_Alice[1], creg1_Alice, creg2_Alice)
mycircuit.barrier()
# Apply classically controlled quantum gates
controlled_ops_Bob_qubit(mycircuit, all_qubits_Bob[0], creg1_Alice, creg2_Alice)

### Look at the complete circuit
mycircuit.draw()

/home/sushmitha/anaconda3/lib/python3.9/site-packages/qiskit/circuit/register.py:121: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not isinstance(key, (int, np.int, np.int32, np.int64, slice, list)):
/home/sushmitha/anaconda3/lib/python3.9/site-packages/qiskit/circuit/register.py:121: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the

┌─────────────────────────────┐ ░            ░      ┌───┐ ░ ┌─┐    ░ »
q10_0: ┤ initialize(0.83666,0.54772) ├─░────────────░───■──┤ H ├─░─┤M├────░─»
       └─────────────────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░ »
q10_1: ────────────────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░─»
                                       ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ »
q11_0: ────────────────────────────────░──────┤ X ├─░────────────░──╫──╫──░─»
                                       ░      └───┘ ░            ░  ║  ║  ░ »
c10: 1/═════════════════════════════════════════════════════════════╩══╬════»
                                                                    0  ║    »
c11: 1/════════════════════════════════════════════════════════════════╩════»
                                                                       0    »
«                     
«q10_0: ──────────────
«                     
«q10_1: ──────────────
«        ┌───┐  ┌───┐ 
«q11_0: ─┤ X ├──┤ Z ├─
«        └─┬─┘  └─┬─┘ 
«          │   ┌──┴──┐
«c10: 1/═══╪═══╡ = 1 ╞
«       ┌──┴──┐└─────┘
«c11: 1/╡ = 1 ╞═══════
«       └─────┘